# initialization

In [1]:
%run 050-init.ipynb

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.4.1
      /_/
                        
Using Scala version 2.12.17, OpenJDK 64-Bit Server VM, 17.0.8
Branch HEAD
Compiled by user centos on 2023-06-19T23:01:01Z
Revision 6b1ff22dde1ead51cbf370be6e48a802daae58b6
Url https://github.com/apache/spark
Type --help for more information.
--2023-08-29 17:45:10--  https://raw.githubusercontent.com/meddash-cloud/meddash-public-datasets/main/data/Ecommerce_Customers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86871 (85K) [text/plain]
Saving to: ‘Ecommerce_Customers.csv.1’

Ecommerce_Customers 100%[===================>]  84.83K  --.-KB/s    in 0.01s   

2023-08-29 17:45:10 (6.62 

# Linear Regression Example PySpark

In [2]:
from pyspark.sql import SparkSession


In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
# Use Spark to read in the Ecommerce Customers csv file.


In [5]:
# Print the Schema of the DataFrame
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [6]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [7]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [8]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


## Setting Up DataFrame for Machine Learning 

In [9]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [10]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [11]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [12]:
output = assembler.transform(data)

In [13]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [14]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [15]:
final_data = output.select("features",'Yearly Amount Spent')

In [16]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [17]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                332|
|   mean| 498.38995111467034|
| stddev|  80.19914143911963|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [18]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                168|
|   mean|  501.1402104715764|
| stddev|  77.74220500224173|
|    min|  275.9184206503857|
|    max|  708.9351848669818|
+-------+-------------------+



In [19]:
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [20]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data,)

In [21]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.96226981845229,38.2245002392917,1.1828126283116907,61.27712603369157] Intercept: -1080.2311457408557


In [22]:
test_results = lrModel.evaluate(test_data)

In [23]:
# Interesting results....
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -4.283277180230982|
| -4.360533736588934|
| -5.948133820971179|
|0.18451524183899437|
|-2.0351222745050563|
|  24.44908510942605|
|  4.186724445492587|
| 3.2791598660969044|
|  -8.45193170017103|
| -6.272899242869755|
| -5.103663127604591|
|-12.667458617604552|
| 18.980604245402674|
|  7.038309277895621|
|-0.3616270628054963|
| 3.7726717921032673|
| -2.021475139901895|
|-0.9025185439294887|
| -5.967991474218195|
| -9.257177164972688|
+-------------------+
only showing top 20 rows



In [24]:
unlabeled_data = test_data.select('features')

In [25]:
predictions = lrModel.transform(unlabeled_data)

In [26]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...| 286.7545229001455|
|[31.0613251567161...|491.91599179449054|
|[31.1280900496166...| 563.2008205680258|
|[31.2606468698795...| 421.1421160151124|
|[31.2681042107507...|  425.505655448329|
|[31.2834474760581...| 567.3320043162414|
|[31.3091926408918...|428.53399339444104|
|[31.4459724827577...|481.59780506903166|
|[31.4474464941278...|427.05467379539505|
|[31.5171218025062...|282.19131989325547|
|[31.5257524169682...| 449.0692899374865|
|[31.5741380228732...| 557.0767307781914|
|[31.6005122003032...|460.19224724569426|
|[31.6548096756927...| 468.2251144496529|
|[31.6610498227460...|416.71998064270633|
|[31.7366356860502...|493.16077446342865|
|[31.8124825597242...| 394.8318201236991|
|[31.8186165667690...|447.32119191406514|
|[31.8745516945853...| 398.2532357204857|
|[31.8854062999117...| 399.3604501374482|
+--------------------+------------

In [28]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 9.818568509526852
MSE: 96.40428757627235
